In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
import time

class CNN2D(nn.Module):
    def __init__(self):
        super(CNN2D, self).__init__()
        self.conv_layer = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(3, 3))
        self.max_pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a channel dimension (channels=1) for the 2D CNN
        x = self.conv_layer(x)
        x = self.max_pool(x)
        x = self.flatten(x)
        return x


class FNN(nn.Module):    
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class RNN(nn.Module):    
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = self.output_layer(out[:, -1, :])
        return out


class LSTM(nn.Module):    
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.output_layer(out[:, -1, :])
        return out


class Trans(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(Trans, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.relu = nn.ReLU()
        self.transformer = nn.Transformer(d_model=32, nhead=4, num_encoder_layers=num_layers)
        self.fc = nn.Linear(32, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.transformer(x, x)  # Self-attention
        x = self.fc(x)
        return x

def switch(mode, input_size, hidden_size, num_layers, output_size):
    switch_dict = {
        1: FNN,
        2: RNN,
        3: LSTM,
        4: Trans
    }
    return switch_dict.get(mode)(input_size, hidden_size, num_layers, output_size)

kmph = 3
hori = 9
cr = 128
mode = 2 # 1=fnn, 2=rnn, 3=lstm, 4=trans
input_size = 10080#2400 #4960 #10080 #20320 #40800 
hidden_size = 512
num_layers = 2
output_size = cr*hori 
num_filters = 32
kernel_size = 3
pool_kernel_size = 2
pool_stride = 2


# Load data from CSV
csv_file_path = f'dataset/Uma_{cr}_{kmph}.csv'
df = pd.read_csv(csv_file_path)

input_data = df.values 
target_data = input_data.copy()  
input_windows = []
target_windows = []
window_size = 12

for i in range(len(input_data) - window_size - hori):
    input_windows.append(input_data[i:i+window_size])
    target_windows.append(target_data[i+window_size:i+window_size+hori])
    
print(np.shape(input_windows))
print(np.shape(target_windows))

input_windows = torch.tensor(input_windows, dtype=torch.float)
target_windows = torch.tensor(target_windows, dtype=torch.float)

dataset = TensorDataset(input_windows, target_windows)

train_ratio = 0.7
valid_ratio = 0.2
test_ratio = 1 - train_ratio - valid_ratio
train_size = int(train_ratio * len(input_windows))
valid_size = int((train_ratio + valid_ratio) * len(input_windows)) - int(train_ratio * len(input_windows))
test_size = len(input_windows) - int((train_ratio + valid_ratio) * len(input_windows))

print(train_size, valid_size, test_size)
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cnn_2d_net = CNN2D().to(device)
# lstm_net = LSTMNetwork(input_size, hidden_size, num_layers, output_size).to(device)
lstm_net = switch(mode, input_size, hidden_size, num_layers, output_size).to(device)
print(sum(p.numel() for p in lstm_net.parameters() if p.requires_grad))
# print(cnn_2d_net)
# print(lstm_net)
criterion = nn.MSELoss()
optimizer = optim.Adam(list(cnn_2d_net.parameters()) + list(lstm_net.parameters()), lr=0.0001)

num_epochs = 1
avg_loss_tran = np.zeros(100)
for epoch in range(num_epochs):
    cnn_2d_net.train()
    lstm_net.train()

    total_loss = 0.0
    cnt = 0
    for inp, targets in train_loader:
        if cnt < 198:
#             print(cnt)
            cnt += 1
            inp = inp.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            
            output_2d = cnn_2d_net(inp)
            output_lstm = lstm_net(output_2d)
            output_lstm = torch.reshape(output_lstm, (32,hori,-1))
#             print(output_lstm.size())
            loss = (criterion(output_lstm, targets))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

    avg_loss_tran[epoch] = np.sqrt(total_loss / len(train_loader))
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss_tran[epoch]}")

    # Validation loop
    cnn_2d_net.eval()
    lstm_net.eval()
    total_time = 0.0
    with torch.no_grad():
        total_val_loss = 0.0
        cnt = 0
        for inp, targets in valid_loader:
            if cnt < 50:
                cnt += 1
                inp = inp.to(device)
                targets = targets.to(device)
#                 print(cnt)

                start_time = time.time()
                output_2d = cnn_2d_net(inp)
                output_lstm = lstm_net(output_2d)
                output_lstm = torch.reshape(output_lstm, (32,hori,-1))
                val_loss = (criterion(output_lstm, targets))
                end_time = time.time()
                
                total_time += end_time - start_time
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(valid_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss}")
    avg_time = total_time * len(valid_loader)
print("Training completed.")
print(avg_time)


(9979, 12, 128)
(9979, 9, 128)
6985 1996 998
6540416
Epoch 1/1, Train Loss: 64.58522120574428
Epoch 1/1, Validation Loss: 3573.181136842758
Training completed.
73.7006471157074


In [20]:
loss_values = [round(avg_loss_tran[epoch], 4) for epoch in range(num_epochs)]
print(loss_values)

[64.5852]


In [21]:
# Assuming you already have the test_dataset from the previous code

# Create DataLoader for the test set
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Test loop
cnn_2d_net.eval()
lstm_net.eval()
#mode = 'trans'
mse_criterion = nn.MSELoss()
total_mse = 0.0

with torch.no_grad():
    cnt = 0
    for inp, targets in test_loader:
        if cnt < 30:
            cnt += 1
            inp = inp.to(device)
            targets = targets.to(device)

            # Pass data sequentially through the networks
            output_2d = cnn_2d_net(inp)
            output_lstm = lstm_net(output_2d)
            output_lstm = torch.reshape(output_lstm, (32,hori,-1))
#             print(cnt)
            mse_loss = torch.sqrt(mse_criterion(output_lstm, targets))
            total_mse += mse_loss.item()
            print("targets", targets.size())
            print("prediction", output_lstm.size())
            
            target = targets.to('cpu')
            predict = output_lstm.to('cpu')
            print(target.size())
#             torch.save(predict[:,:,:],"predict_rnn_5120.pt")
#             torch.save(target[:,:,:],"target_rnn_5120.pt")
            print(predict.size())
#             print(cnt)
            if cnt == 1:
                print(cnt)
                torch.save(predict[:,:,:],f"D:/CMa/{cr}/predict_{mode}_{cr}_3_{hori}.pt")
                torch.save(target[:,:,:],f"D:/CMa/{cr}/target_{mode}_{cr}_3_{hori}.pt")

avg_mse = total_mse / len(test_loader)
print(f"Root Mean Squared Error (RMSE): {avg_mse}")


targets torch.Size([32, 9, 128])
prediction torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
1
targets torch.Size([32, 9, 128])
prediction torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
targets torch.Size([32, 9, 128])
prediction torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
targets torch.Size([32, 9, 128])
prediction torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
targets torch.Size([32, 9, 128])
prediction torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
targets torch.Size([32, 9, 128])
prediction torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
targets torch.Size([32, 9, 128])
prediction torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
targets torch.Size([32, 9, 128])
prediction torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
torch.Size([32, 9, 128])
targets torch.Size([32, 9, 128])
prediction to